In [ ]:
# TFLite & TensorFlow Model Evaluation Pipeline
import os
import sys
import gc
import pickle as pkl
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import balanced_accuracy_score, f1_score, confusion_matrix

# Configuration
DATA_DIR = Path('../../Data/Experiment_Data/2_PreprocessDataset')
MODEL_DIR = Path('../../Models/TFlite_model/Multimodal')
OUTPUT_ACC_DIR = Path('../../Result/Experiment_Result/Model_Accuracy')
OUTPUT_PRED_DIR = Path('../..//Result/Experiment_Result/Model_Preds/Multimodal')
OUTPUT_CM_DIR = Path('../..//Result/Experiment_Result/Confusion_Matrix/Multimodal/NoTimeVoting')

# Environment
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Seeds
SEED = 4
random = np.random.RandomState(SEED)
tf.random.set_seed(SEED)

# Threads
tf.config.threading.set_intra_op_parallelism_threads(4)
tf.config.threading.set_inter_op_parallelism_threads(4)

# Model parameters
MODEL_VERSION = 1
TFLITE_MODEL = MODEL_DIR / f'quan_multimodal_cnn_ver{MODEL_VERSION}.tflite'

# Load normalization params & label binarizer
with open(f'../../Normalization_params/Normalization_params_pickle/normalization_params_Right_ver{MODEL_VERSION}.pkl','rb') as f:
    norm_params = pkl.load(f)
with open('../../LabelBinarizer/Label_binarizer_6_classes.pkl','rb') as f:
    lb = pkl.load(f)
    print('Label mapping:', dict(zip(lb.classes_, lb.transform(lb.classes_))))

# Class names
CLASS_NAMES = ['Shower','Tooth_brushing','Washing_hands','Wiping','Vacuum_Cleaner','Other']

# Utility: batch TFLite inference
def tflite_batch_predict(model_path: Path, pid: str, batch_size: int = 128) -> pd.DataFrame:
    # Load TFLite interpreter
    interpreter = tf.lite.Interpreter(model_path=str(model_path), num_threads=4)
    interp_inputs = interpreter.get_input_details()
    interp_output = interpreter.get_output_details()[0]

    # Resize inputs for batching
    imu_shape = list(interp_inputs[0]['shape'])
    audio_shape = list(interp_inputs[1]['shape'])
    interpreter.resize_tensor_input(interp_inputs[0]['index'], [batch_size]+imu_shape[1:])
    interpreter.resize_tensor_input(interp_inputs[1]['index'], [batch_size]+audio_shape[1:])
    interpreter.allocate_tensors()

    # Data load
    pkl_path = DATA_DIR / pid / f'{pid}_preprocessing.pkl'
    data = pkl.load(open(pkl_path,'rb'))
    imu = data['IMU'].astype(np.float32)
    audio = data['Audio'].astype(np.float32)
    y_true = np.array(data['Activity'])

    # Normalize IMU
    for key in ['max','min','mean','std']:
        norm_params[key] = norm_params[key].astype(np.float32)
    pm,pn,mu,sd = [norm_params[k].reshape(1,1,-1) for k in ('max','min','mean','std')]
    imu = 1 + (imu - pm)*2/(pm - pn)
    imu = (imu - mu)/sd

    # Prepare audio
    audio = np.expand_dims(audio, -1)

    # Predict in batches
    preds = []
    n = imu.shape[0]
    for i in tqdm(range(0,n,batch_size)):
        end = min(i+batch_size, n)
        b_imu = imu[i:end]
        b_audio = audio[i:end]
        # Resize last batch if needed
        if b_imu.shape[0] != batch_size:
            interpreter.resize_tensor_input(interp_inputs[0]['index'], [b_imu.shape[0]]+imu_shape[1:])
            interpreter.resize_tensor_input(interp_inputs[1]['index'], [b_audio.shape[0]]+audio_shape[1:])
            interpreter.allocate_tensors()
        interpreter.set_tensor(interp_inputs[0]['index'], b_imu)
        interpreter.set_tensor(interp_inputs[1]['index'], b_audio)
        interpreter.invoke()
        out = interpreter.get_tensor(interp_output['index'])
        preds.append(out)
    preds = np.vstack(preds)

    # Build DataFrame
    df = pd.DataFrame(preds, columns=lb.classes_)
    df.insert(0, 'y_true', y_true)
    return df

# Main evaluation loop
for pid in sorted(DIR for DIR in os.listdir(DATA_DIR) if (DATA_DIR/DIR).is_dir()):
    print(f'Processing {pid}...')
    # Batch predict
    df_pred = tflite_batch_predict(TFLITE_MODEL, pid)
    df_pred['y_pred'] = df_pred.drop(columns=['y_true']).idxmax(axis=1)

    # Metrics
    ba = balanced_accuracy_score(df_pred['y_true'], df_pred['y_pred'])
    f1 = f1_score(df_pred['y_true'], df_pred['y_pred'], average='weighted')
    # Save accuracy
    OUTPUT_ACC_DIR.mkdir(exist_ok=True, parents=True)
    with open(OUTPUT_ACC_DIR/f'{pid}_accuracy.txt','w') as f:
        f.write(f'Balanced Accuracy: {ba:.4f}\nF1 Score: {f1:.4f}\n')

    # Save predictions
    pred_dir = OUTPUT_PRED_DIR / pid
    pred_dir.mkdir(exist_ok=True, parents=True)
    df_pred.to_csv(pred_dir/f'{pid}_quan_ver{MODEL_VERSION}.csv', index=False)

    # Confusion matrix
    cm = confusion_matrix(df_pred['y_true'], df_pred['y_pred'], labels=CLASS_NAMES)
    cm_pct = (cm.astype(float)/cm.sum(axis=1)[:,None])*100
    cm_pct = np.nan_to_num(cm_pct)

    fig, ax = plt.subplots(figsize=(12,10))
    im = ax.imshow(cm_pct, cmap='Greens', vmin=0, vmax=100)
    ax.set_xticks(range(len(CLASS_NAMES)))
    ax.set_xticklabels(CLASS_NAMES, rotation=45, ha='right')
    ax.set_yticks(range(len(CLASS_NAMES)))
    ax.set_yticklabels(CLASS_NAMES)
    thresh = cm_pct.max()/2
    for i in range(cm_pct.shape[0]):
        for j in range(cm_pct.shape[1]):
            color = 'white' if cm_pct[i,j]>thresh else 'black'
            ax.text(j,i,f'{cm_pct[i,j]:.1f}', ha='center', va='center', color=color, weight='bold')
    ax.set_title('Confusion Matrix without TimeVoting')
    plt.colorbar(im, ax=ax)
    plt.tight_layout()

    # Save image
    save_cm = OUTPUT_CM_DIR / pid
    save_cm.mkdir(exist_ok=True, parents=True)
    fig.savefig(save_cm/f'{pid}_NoTimeVoting_Confusion.png')
    plt.close(fig)
    gc.collect()